In [10]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
import spacy
from models.SIM import SIM
import utils
import pickle
from sympy.parsing.sympy_parser import parse_expr
from sympy.parsing.sympy_parser import standard_transformations, convert_xor,implicit_multiplication

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
math_train = pd.read_json(r"..\Data\dolphin-number_word_std\number_word_std.dev.json")
math_test = pd.read_json(r"..\Data\dolphin-number_word_std\number_word_std.test.json")

In [7]:
model = SIM()
model.fit(math_train)

SIM()

In [8]:
model.score(math_train)

0.17379679144385027

In [8]:
math_eq_format=['unkn: d, g', 'equ: a+b+c+d=17', 'equ: d+m+f+g=17', 'equ: g+h+i=17', 'equ: a = 4', 'equ: be_different(a,b,c,d,m,f,g,h,i)', 'equ: a >= 1', 'equ: a <= 9', 'equ: b >= 1', 'equ: b <= 9', 'equ: c >= 1', 'equ: c <= 9', 'equ: d >= 1', 'equ: d <= 9', 'equ: m >= 1', 'equ: m <= 9', 'equ: f >= 1', 'equ: f <= 9', 'equ: g >= 1', 'equ: g <= 9', 'equ: h >= 1', 'equ: h <= 9', 'equ: i >= 1', 'equ: i <= 9']
math_eq_format

['unkn: d, g',
 'equ: a+b+c+d=17',
 'equ: d+m+f+g=17',
 'equ: g+h+i=17',
 'equ: a = 4',
 'equ: be_different(a,b,c,d,m,f,g,h,i)',
 'equ: a >= 1',
 'equ: a <= 9',
 'equ: b >= 1',
 'equ: b <= 9',
 'equ: c >= 1',
 'equ: c <= 9',
 'equ: d >= 1',
 'equ: d <= 9',
 'equ: m >= 1',
 'equ: m <= 9',
 'equ: f >= 1',
 'equ: f <= 9',
 'equ: g >= 1',
 'equ: g <= 9',
 'equ: h >= 1',
 'equ: h <= 9',
 'equ: i >= 1',
 'equ: i <= 9']

In [ ]:
TRANSFORMATION = standard_transformations + (implicit_multiplication,convert_xor,)
kw_parser = dict(evaluate=True,transformations=TRANSFORMATION)

True

In [11]:
to_wolfram=True
parse_eq_list = []
for eq in math_eq_format[1:]:
    # remove whitespaces and move to onesided equation
    clean_eq = eq.split('equ: ')[-1].replace(' ','')
    if '<=' in clean_eq:
        rhs,lhs = clean_eq.split('<=')
        parse_eq_list.append()
    else:
        to_wolfram=False
        rhs,lhs = clean_eq.split('=')
        parse_eq_list += [parse_expr(lhs, **kw_parser) * -1 + parse_expr(rhs, **kw_parser)]

NameError: name 'kw_parser' is not defined